# Global Patterns of Inequality in Urban Exposure to Climate Hazards
## Reproducible Analysis Notebook

**Authors:** Sidique Gawusu, Benjamin K. Sovacool, Xiaobing Zhang, Oisik Das, Seidu Abdulai Jamatutu, Abubakari Ahmed, Imelda Piri

**Journal:** *Nature Cities*

**Dataset:** GHS Urban Centre Database 2025 (GHS-UCDB R2024A V1.1)  
**DOI:** [10.2905/1a338be6-7eaf-480c-9664-3a8ade88cbcd](https://doi.org/10.2905/1a338be6-7eaf-480c-9664-3a8ade88cbcd)

---

### How to reproduce

1. Download the GHS-UCDB R2024A V1.1 GeoPackage (~1.2 GB) from the DOI link above
2. Place the `.gpkg` file in your working directory (or update `DATA_PATH` in Cell 3)
3. Install dependencies: `pip install numpy pandas matplotlib seaborn scipy statsmodels geopandas pyogrio openpyxl`
4. Run all cells sequentially
5. Outputs are saved to `figures/` (PDF + PNG) and `tables/` (CSV + XLSX) directories

### Output inventory

| Cell | Output | Manuscript location |
|------|--------|-------------------|
| 5 | Descriptive statistics | Table 1 |
| 6 | Multi-hazard exposure distributions | Figure 1 |
| 7 | LECZ–HDI scatter + regression residuals | Figure 2 |
| 8 | Hazard × HDI bivariate heatmap | Figure 3 |
| 9 | Demographic composition | Figure 4 |
| 10 | Quantile regression coefficients | Figure 5 |
| 11 | OLS regression results (Models 1–4) | Extended Data Table 1 |
| 12 | Regional disaggregation | Extended Data |
| 13 | Top 50 exposed low-income cities | Extended Data Table 3 |
| 14 | Sensitivity analyses | Extended Data Table 2 |
| 15 | Spearman correlation matrix | Extended Data Figure 3 |

## Cell 1: Environment Setup
Install and import all required packages. The analysis requires Python >= 3.9 with standard scientific computing libraries plus `geopandas` for GeoPackage I/O.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import matplotlib.patheffects as pe
from matplotlib.colors import LinearSegmentedColormap, Normalize, BoundaryNorm
from matplotlib.lines import Line2D
import seaborn as sns
from scipy import stats
from scipy.stats import kruskal, mannwhitneyu, spearmanr, pearsonr
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.quantile_regression import QuantReg
from statsmodels.nonparametric.smoothers_lowess import lowess
from collections import OrderedDict
import os

try:
    import geopandas as gpd
except ImportError:
    os.system('pip install geopandas pyogrio --quiet')
    import geopandas as gpd

print(f"NumPy {np.__version__}, Pandas {pd.__version__}, Matplotlib {matplotlib.__version__}")
print("All packages loaded.")

## Cell 2: Nature Cities Figure Style
Configure matplotlib to produce publication-quality figures conforming to Nature portfolio specifications (89 mm single column, 183 mm double column). Uses the Wong (2011) colourblind-safe palette for income group encoding.

In [ ]:
MM_TO_INCH = 1 / 25.4
NATURE_SINGLE_COL = 89 * MM_TO_INCH
NATURE_DOUBLE_COL = 183 * MM_TO_INCH
NATURE_MAX_HEIGHT = 247 * MM_TO_INCH
FIGURE_DPI = 300

WONG_PALETTE = {
    'black': '#000000', 'orange': '#E69F00', 'sky_blue': '#56B4E9',
    'green': '#009E73', 'yellow': '#F0E442', 'blue': '#0072B2',
    'vermilion': '#D55E00', 'purple': '#CC79A7',
}

INCOME_COLORS = OrderedDict([
    ('Low',          WONG_PALETTE['vermilion']),
    ('Lower Middle', WONG_PALETTE['orange']),
    ('Upper Middle', WONG_PALETTE['sky_blue']),
    ('High',         WONG_PALETTE['blue']),
])
INCOME_ORDER = ['Low', 'Lower Middle', 'Upper Middle', 'High']

plt.rcParams.update({
    'font.family': 'sans-serif',
    'font.sans-serif': ['Helvetica', 'Arial', 'DejaVu Sans'],
    'font.size': 7, 'axes.labelsize': 7, 'axes.titlesize': 8,
    'axes.linewidth': 0.5, 'axes.labelpad': 3,
    'axes.spines.top': False, 'axes.spines.right': False,
    'xtick.labelsize': 6, 'ytick.labelsize': 6,
    'xtick.major.size': 3, 'xtick.major.width': 0.5,
    'ytick.major.size': 3, 'ytick.major.width': 0.5,
    'xtick.direction': 'out', 'ytick.direction': 'out',
    'legend.fontsize': 6, 'legend.frameon': False,
    'lines.linewidth': 0.8, 'lines.markersize': 3,
    'figure.dpi': 150, 'savefig.dpi': FIGURE_DPI,
    'savefig.bbox': 'tight', 'savefig.pad_inches': 0.02,
    'grid.linewidth': 0.3, 'grid.alpha': 0.3,
})


def add_panel_label(ax, label, x=-0.12, y=1.08):
    """Add bold panel label (a, b, c...) to subplot."""
    ax.text(x, y, label, transform=ax.transAxes,
            fontsize=8, fontweight='bold', va='top', ha='left')


def save_figure(fig, filename, formats=('pdf', 'png')):
    """Save figure in multiple formats to figures/ directory."""
    os.makedirs('figures', exist_ok=True)
    for fmt in formats:
        fig.savefig(f'figures/{filename}.{fmt}', format=fmt, dpi=FIGURE_DPI,
                    bbox_inches='tight', pad_inches=0.02,
                    facecolor='white', edgecolor='none')
    print(f"  Saved: {', '.join(f'figures/{filename}.{f}' for f in formats)}")


print("Nature Cities style configured.")

## Cell 3: Data Loading
Load six thematic layers from the GHS-UCDB R2024A GeoPackage:
- **General Characteristics:** city name, country, area, population, income group, SDG region
- **Socioeconomic:** HDI, GDP, life expectancy, demographics
- **Exposure:** flood, LECZ, coastal, earthquake, cyclone
- **Emissions:** PM₂.₅, CO₂
- **Health:** hospital access
- **Infrastructure:** CISI (Critical Infrastructure Spatial Index)

**Note:** Update `DATA_PATH` to point to your local copy of the GeoPackage. Google Colab users should uncomment the drive mount lines.

In [ ]:
# --- Google Colab users: uncomment the next two lines ---
# from google.colab import drive
# drive.mount('/content/drive')

# --- Set path to GHS-UCDB GeoPackage ---
DATA_PATH = 'GHS_UCDB_GLOBE_R2024A.gpkg'  # UPDATE THIS PATH

if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(
        f"GeoPackage not found at {DATA_PATH}.\n"
        "Download from: https://data.jrc.ec.europa.eu/dataset/"
        "1a338be6-7eaf-480c-9664-3a8ade88cbcd"
    )

print(f"File: {DATA_PATH} ({os.path.getsize(DATA_PATH)/1e6:.1f} MB)")

LAYERS_TO_LOAD = {
    'gc':   'GHS_UCDB_THEME_GENERAL_CHARACTERISTICS_GLOBE_R2024A',
    'sc':   'GHS_UCDB_THEME_SOCIOECONOMIC_GLOBE_R2024A',
    'ex':   'GHS_UCDB_THEME_EXPOSURE_GLOBE_R2024A',
    'em':   'GHS_UCDB_THEME_EMISSIONS_GLOBE_R2024A',
    'hl':   'GHS_UCDB_THEME_HEALTH_GLOBE_R2024A',
    'infr': 'GHS_UCDB_THEME_INFRASTRUCTURES_GLOBE_R2024A',
}


def strip_bom(text):
    """Remove Unicode BOM characters from strings."""
    if isinstance(text, str):
        return text.replace('\ufeff', '').strip()
    return text


def load_gpkg_layer(path, layer_name, keep_geometry=False):
    """Load a single layer from the GeoPackage, stripping BOM artifacts."""
    gdf_layer = gpd.read_file(path, layer=layer_name)
    gdf_layer.columns = [strip_bom(c) for c in gdf_layer.columns]
    for col in gdf_layer.select_dtypes(include=['object']).columns:
        if col != 'geometry':
            gdf_layer[col] = gdf_layer[col].apply(strip_bom)
    if not keep_geometry and 'geometry' in gdf_layer.columns:
        gdf_layer = pd.DataFrame(gdf_layer.drop(columns='geometry'))
    return gdf_layer


print("\nLoading layers...\n")
raw = {}
for key, layer_name in LAYERS_TO_LOAD.items():
    print(f"  {key:5s} -> {layer_name}...", end=' ', flush=True)
    raw[key] = load_gpkg_layer(DATA_PATH, layer_name, keep_geometry=(key == 'gc'))
    print(f"({raw[key].shape[0]:,} x {raw[key].shape[1]:,})")

print(f"\nAll layers loaded. Urban centres: {raw['gc'].shape[0]:,}")

## Cell 4: Data Preparation
Merge all thematic layers on the urban centre identifier (`ID_UC_G0`), standardise income group labels to four World Bank categories, compute derived variables, and construct the **Compound Hazard Index (CHI)** as the arithmetic mean of five min-max normalised exposure dimensions:
1. Riverine flood (100-year return period)
2. Low-elevation coastal zone (< 10 m)
3. Coastal flood (100-year return period)
4. Earthquake (≥ MMI 6)
5. Tropical cyclone (100-year return period)

Final analytical sample: **n = 11,413** urban centres across four income groups.

In [ ]:
ID_COL = 'ID_UC_G0'


def find_col(dataframe, pattern, exclude=None):
    """Find column matching pattern (case-insensitive), excluding specified substrings."""
    for c in sorted(dataframe.columns, reverse=True):
        if pattern.upper() in c.upper():
            if exclude and any(ex.upper() in c.upper() for ex in exclude):
                continue
            return c
    return None


# --- General Characteristics ---
gc = raw['gc']
gc_cols = {
    'name': find_col(gc, 'GC_UCN_MAI'),
    'country_gadm': find_col(gc, 'GC_CNT_GAD'),
    'country_un': find_col(gc, 'GC_CNT_UNN'),
    'area_km2': find_col(gc, 'GC_UCA_KM2'),
    'population': find_col(gc, 'GC_POP_TOT'),
    'income_group': find_col(gc, 'GC_DEV_WIG'),
    'sdg_region': find_col(gc, 'GC_DEV_USR'),
}

gc_select = [ID_COL] + [v for v in gc_cols.values() if v]
gc_rename = {ID_COL: 'id'}
gc_rename.update({v: k for k, v in gc_cols.items() if v})
df = gc[gc_select + ['geometry']].copy().rename(columns=gc_rename)

# --- Socioeconomic ---
sc = raw['sc']
sc_map = {
    'hdi':           find_col(sc, 'SC_SEC_HDI', exclude=['GDF', 'GDM']),
    'gdp_avg':       find_col(sc, 'SC_GDP_AVG'),
    'gdp_total':     find_col(sc, 'SC_GDP_SUM'),
    'life_exp':      find_col(sc, 'SC_SEC_LET', exclude=['LEF', 'LEM']),
    'schooling_exp': find_col(sc, 'SC_SEC_SET', exclude=['SEF', 'SEM']),
    'pct_young':     find_col(sc, 'SC_SEC_PCY'),
    'pct_adult':     find_col(sc, 'SC_SEC_PCA'),
    'pct_old':       find_col(sc, 'SC_SEC_PCO'),
    'pct_female':    find_col(sc, 'SC_SEC_PCF'),
    'pct_male':      find_col(sc, 'SC_SEC_PCM'),
    'gdi':           find_col(sc, 'SC_SEC_GDI', exclude=['GDF', 'GDM']),
}
sc_r = {v: k for k, v in sc_map.items() if v}
df = df.merge(sc[[ID_COL]+list(sc_r.keys())].rename(columns={**sc_r, ID_COL:'id'}),
              on='id', how='left')
print(f"Socioeconomic: {len(sc_r)}/{len(sc_map)} variables matched")

# --- Exposure ---
ex = raw['ex']
ex_map = {
    'pop_flood_100yr':            find_col(ex, 'EX_100_POP'),
    'pop_flood_10yr':             find_col(ex, 'EX_010_POP'),
    'share_pop_flood_100yr':      find_col(ex, 'EX_100_SHP'),
    'share_pop_flood_10yr':       find_col(ex, 'EX_010_SHP'),
    'pop_lecz_5m':                find_col(ex, 'EX_L05_POP'),
    'pop_lecz_10m':               find_col(ex, 'EX_L10_POP'),
    'share_pop_lecz_5m':          find_col(ex, 'EX_L05_SHP'),
    'share_pop_lecz_10m':         find_col(ex, 'EX_L10_SHP'),
    'share_pop_lecz_below10m':    find_col(ex, 'EX_LEC_SHP'),
    'builtup_lecz_5m':            find_col(ex, 'EX_L05_BUS', exclude=['B1', 'B2']),
    'pop_coastal_100yr':          find_col(ex, 'EX_CF1_POP'),
    'pop_coastal_20yr':           find_col(ex, 'EX_CF2_POP'),
    'share_pop_coastal_100yr':    find_col(ex, 'EX_CF1_SHP'),
    'share_pop_coastal_20yr':     find_col(ex, 'EX_CF2_SHP'),
    'share_pop_earthquake_mmi6':  find_col(ex, 'EX_SHA_POP'),
    'pop_cyclone_100yr':          find_col(ex, 'EX_100_S1P'),
    'pop_cyclone_400yr_ss3':      find_col(ex, 'EX_400_S3P'),
}
ex_r = {v: k for k, v in ex_map.items() if v}
df = df.merge(ex[[ID_COL]+list(ex_r.keys())].rename(columns={**ex_r, ID_COL:'id'}),
              on='id', how='left')
print(f"Exposure: {len(ex_r)}/{len(ex_map)} variables matched")

# --- Emissions ---
em = raw['em']
em_map = {
    'pm25_deaths':    find_col(em, 'EM_PM2_MOR'),
    'pm25_conc':      find_col(em, 'EM_PM2_CON'),
    'co2_total':      find_col(em, 'EM_CO2_TOT'),
    'co2_per_capita': find_col(em, 'EM_CO2_PEC'),
}
em_r = {v: k for k, v in em_map.items() if v}
if em_r:
    df = df.merge(em[[ID_COL]+list(em_r.keys())].rename(columns={**em_r, ID_COL:'id'}),
                  on='id', how='left')
print(f"Emissions: {len(em_r)}/{len(em_map)} variables matched")

# --- Health ---
hl = raw['hl']
hl_map = {
    'hospitals_per_cap':       find_col(hl, 'HL_FPC_HOS'),
    'share_pop_near_hospital': find_col(hl, 'HL_SHP_HOS'),
}
hl_r = {v: k for k, v in hl_map.items() if v}
if hl_r:
    df = df.merge(hl[[ID_COL]+list(hl_r.keys())].rename(columns={**hl_r, ID_COL:'id'}),
                  on='id', how='left')
print(f"Health: {len(hl_r)}/{len(hl_map)} variables matched")

# --- Infrastructure ---
infr = raw['infr']
infr_map = {
    'cisi_all':    find_col(infr, 'IN_CIS_ALL'),
    'cisi_energy': find_col(infr, 'IN_CIS_ENE'),
}
infr_r = {v: k for k, v in infr_map.items() if v}
if infr_r:
    df = df.merge(infr[[ID_COL]+list(infr_r.keys())].rename(columns={**infr_r, ID_COL:'id'}),
                  on='id', how='left')
print(f"Infrastructure: {len(infr_r)}/{len(infr_map)} variables matched")

# --- Numeric conversion ---
skip = {'id', 'name', 'country_gadm', 'country_un', 'income_group', 'sdg_region', 'geometry'}
for col in df.columns:
    if col not in skip:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# --- Standardise income group labels ---
income_map = {
    'Low': 'Low', 'Low Income': 'Low', 'Low income': 'Low',
    'Lower Middle': 'Lower Middle', 'Lower Middle Income': 'Lower Middle',
    'Lower middle income': 'Lower Middle', 'Lower-middle income': 'Lower Middle',
    'Upper Middle': 'Upper Middle', 'Upper Middle Income': 'Upper Middle',
    'Upper middle income': 'Upper Middle', 'Upper-middle income': 'Upper Middle',
    'High': 'High', 'High Income': 'High', 'High income': 'High',
}
df['income_group'] = df['income_group'].map(
    lambda x: income_map.get(str(x).strip(), str(x).strip()) if pd.notna(x) else np.nan
)
print(f"\nIncome groups after standardisation:")
print(df['income_group'].value_counts(dropna=False))

# --- Derived variables ---
df['pop_density'] = df['population'] / df['area_km2'].replace(0, np.nan)

if 'pop_cyclone_100yr' in df.columns:
    df['share_pop_cyclone_100yr'] = (
        df['pop_cyclone_100yr'] / df['population'].replace(0, np.nan) * 100)
else:
    df['share_pop_cyclone_100yr'] = np.nan

df['pop_lecz_total'] = df.get('pop_lecz_5m', pd.Series(0, index=df.index)).fillna(0) + \
                        df.get('pop_lecz_10m', pd.Series(0, index=df.index)).fillna(0)

# --- Compound Hazard Exposure Index (arithmetic mean of min-max normalised dimensions) ---
hazard_vars = ['share_pop_flood_100yr', 'share_pop_lecz_below10m',
               'share_pop_coastal_100yr', 'share_pop_earthquake_mmi6',
               'share_pop_cyclone_100yr']

available_hazards = [v for v in hazard_vars if v in df.columns and df[v].notna().sum() > 100]
print(f"\nCompound Hazard Index components ({len(available_hazards)}/{len(hazard_vars)}):")

for var in available_hazards:
    vals = df[var]
    vmin, vmax = vals.min(), vals.max()
    if pd.notna(vmin) and pd.notna(vmax) and vmax > vmin:
        df[f'{var}_norm'] = (vals - vmin) / (vmax - vmin)
    else:
        df[f'{var}_norm'] = 0.0
    print(f"  {var} (n={vals.notna().sum():,})")

norm_cols = [f'{v}_norm' for v in available_hazards]
df['compound_hazard_index'] = df[norm_cols].mean(axis=1, skipna=True)

# --- Quintile classifications ---
try:
    df['hdi_quintile'] = pd.qcut(df['hdi'], q=5,
        labels=['Q1\n(Lowest)', 'Q2', 'Q3', 'Q4', 'Q5\n(Highest)'], duplicates='drop')
except Exception:
    df['hdi_quintile'] = pd.qcut(df['hdi'].rank(method='first'), q=5,
        labels=['Q1\n(Lowest)', 'Q2', 'Q3', 'Q4', 'Q5\n(Highest)'])
try:
    df['hazard_quintile'] = pd.qcut(df['compound_hazard_index'], q=5,
        labels=['Q1\n(Lowest)', 'Q2', 'Q3', 'Q4', 'Q5\n(Highest)'], duplicates='drop')
except Exception:
    df['hazard_quintile'] = pd.qcut(df['compound_hazard_index'].rank(method='first'), q=5,
        labels=['Q1\n(Lowest)', 'Q2', 'Q3', 'Q4', 'Q5\n(Highest)'])

# --- Final analytical sample ---
df_valid = df[df['income_group'].isin(INCOME_ORDER)].copy()

print(f"\n{'='*60}")
print(f"FINAL ANALYTICAL SAMPLE: {df_valid.shape[0]:,} urban centres")
print(f"{'='*60}")
for grp in INCOME_ORDER:
    print(f"  {grp:20s} {(df_valid['income_group'] == grp).sum():>6,}")

print(f"\nKey variable coverage (% non-missing):")
for v in ['hdi', 'gdp_avg', 'share_pop_flood_100yr', 'share_pop_lecz_below10m',
           'share_pop_coastal_100yr', 'share_pop_earthquake_mmi6',
           'pct_young', 'pct_old', 'pm25_conc', 'cisi_all', 'compound_hazard_index']:
    if v in df_valid.columns:
        print(f"  {v:35s} {df_valid[v].notna().mean()*100:6.1f}%")

## Cell 5: Table 1 — Descriptive Statistics
Median (IQR) for all key variables stratified by World Bank income group. Kruskal-Wallis tests assess between-group differences. Corresponds to **Table 1** in the manuscript.

In [ ]:
def create_table1(data, income_order):
    """Generate Table 1: Descriptive statistics by World Bank income group."""
    variables = OrderedDict([
        ('population',              ('Population',                          ',.0f')),
        ('area_km2',                ('Area (km²)',                          ',.1f')),
        ('pop_density',             ('Pop. density (per km²)',              ',.0f')),
        ('hdi',                     ('HDI',                                 '.3f')),
        ('gdp_avg',                 ('GDP per capita (PPP)',                ',.0f')),
        ('life_exp',                ('Life expectancy (yr)',                '.1f')),
        ('schooling_exp',           ('Expected schooling (yr)',             '.1f')),
        ('pct_young',               ('Pop. 0–14 (%)',                      '.1f')),
        ('pct_old',                 ('Pop. >65 (%)',                        '.1f')),
        ('pct_female',              ('Female pop. (%)',                     '.1f')),
        ('share_pop_flood_100yr',   ('Pop. 100-yr flood (%)',              '.1f')),
        ('share_pop_lecz_below10m', ('Pop. LECZ <10 m (%)',               '.1f')),
        ('share_pop_coastal_100yr', ('Pop. coastal flood (%)',             '.1f')),
        ('share_pop_earthquake_mmi6', ('Pop. earthquake ≥MMI 6 (%)',      '.1f')),
        ('share_pop_cyclone_100yr', ('Pop. cyclone (%)',                   '.1f')),
        ('compound_hazard_index',   ('Compound Hazard Index',              '.3f')),
        ('pm25_conc',               ('PM₂.₅ (µg/m³)',                     '.1f')),
        ('cisi_all',                ('CISI',                               '.3f')),
        ('share_pop_near_hospital', ('Pop. near hospital (%)',             '.1f')),
    ])
    variables = OrderedDict((k, v) for k, v in variables.items() if k in data.columns)

    rows = [{'Variable': 'N (urban centres)',
             **{g: f"{(data['income_group']==g).sum():,}" for g in income_order},
             'All': f"{data.shape[0]:,}", 'p-value': ''}]

    for var_name, (label, fmt) in variables.items():
        row = {'Variable': label}
        gvals = []
        for grp in income_order:
            gd = data.loc[data['income_group'] == grp, var_name].dropna()
            gvals.append(gd)
            if len(gd) > 0:
                m, q1, q3 = gd.median(), gd.quantile(0.25), gd.quantile(0.75)
                row[grp] = f"{m:{fmt}} ({q1:{fmt}}–{q3:{fmt}})"
            else:
                row[grp] = '—'
        ad = data[var_name].dropna()
        if len(ad) > 0:
            m, q1, q3 = ad.median(), ad.quantile(0.25), ad.quantile(0.75)
            row['All'] = f"{m:{fmt}} ({q1:{fmt}}–{q3:{fmt}})"
        else:
            row['All'] = '—'
        vg = [g for g in gvals if len(g) >= 5]
        if len(vg) >= 2:
            try:
                _, p = kruskal(*vg)
                row['p-value'] = '<0.001' if p < 0.001 else f"{p:.3f}"
            except Exception:
                row['p-value'] = '—'
        else:
            row['p-value'] = '—'
        rows.append(row)

    return pd.DataFrame(rows)[['Variable'] + income_order + ['All', 'p-value']]


table1 = create_table1(df_valid, INCOME_ORDER)
print("\n" + "="*100)
print("TABLE 1: Descriptive statistics by World Bank income group")
print("="*100)
print(table1.to_string(index=False))
os.makedirs('tables', exist_ok=True)
table1.to_csv('tables/Table1_descriptive_stats.csv', index=False)
table1.to_excel('tables/Table1_descriptive_stats.xlsx', index=False)
print("  Saved: tables/Table1_descriptive_stats.csv/.xlsx")

## Cell 6: Figure 1 — Multi-Hazard Exposure Distributions
Box plots with jittered data points showing the distribution of four hazard dimensions and the Compound Hazard Index across income groups. Mann-Whitney U tests compare Low vs High income groups. Corresponds to **Figure 1** in the manuscript.

In [ ]:
def figure1_multihazard(data, income_order, income_colors):
    """Figure 1: Multi-hazard exposure distributions by income group."""
    fig = plt.figure(figsize=(NATURE_DOUBLE_COL, NATURE_DOUBLE_COL * 0.65))
    gs = gridspec.GridSpec(2, 2, hspace=0.45, wspace=0.35,
                           left=0.08, right=0.97, top=0.93, bottom=0.08)
    panels = {
        'a': ('share_pop_flood_100yr', 'Population in 100-yr\nflood zone (%)'),
        'b': ('share_pop_lecz_below10m', 'Population in\nLECZ <10 m (%)'),
        'c': ('share_pop_earthquake_mmi6', 'Population exposed to\nearthquake ≥MMI 6 (%)'),
        'd': ('compound_hazard_index', 'Compound Hazard\nIndex (0–1)'),
    }
    for idx, (label, (var, ylabel)) in enumerate(panels.items()):
        ax = fig.add_subplot(gs[idx // 2, idx % 2])
        if var not in data.columns or data[var].notna().sum() < 10:
            ax.text(0.5, 0.5, f'{var}\nnot available', transform=ax.transAxes,
                   ha='center', va='center', fontsize=6, color='grey')
            add_panel_label(ax, label); continue

        plot_data = [data.loc[data['income_group'] == g, var].dropna() for g in income_order]
        bp = ax.boxplot(plot_data, positions=range(len(income_order)), widths=0.6,
            patch_artist=True, showfliers=False, showmeans=True,
            meanprops=dict(marker='D', markerfacecolor='white',
                          markeredgecolor='black', markersize=2.5, markeredgewidth=0.5),
            medianprops=dict(color='black', linewidth=0.8),
            whiskerprops=dict(linewidth=0.5, color='#333333'),
            capprops=dict(linewidth=0.5, color='#333333'),
            boxprops=dict(linewidth=0.5))

        for patch, grp in zip(bp['boxes'], income_order):
            patch.set_facecolor(income_colors[grp]); patch.set_alpha(0.75)

        rng = np.random.default_rng(42)
        for i, grp in enumerate(income_order):
            vals = data.loc[data['income_group'] == grp, var].dropna()
            n_plot = min(300, len(vals))
            if n_plot > 0:
                vs = vals.sample(n=n_plot, random_state=42) if len(vals) > n_plot else vals
                ax.scatter(i + rng.normal(0, 0.08, len(vs)), vs, s=0.3, alpha=0.15,
                          color=income_colors[grp], zorder=1, rasterized=True)

        ax.set_xticks(range(len(income_order)))
        short_labels = ['Low', 'Lower\nMiddle', 'Upper\nMiddle', 'High']
        ax.set_xticklabels(short_labels, rotation=0, ha='center')
        ax.set_ylabel(ylabel)

        gl = data.loc[data['income_group'] == 'Low', var].dropna()
        gh = data.loc[data['income_group'] == 'High', var].dropna()
        if len(gl) > 5 and len(gh) > 5:
            _, pv = mannwhitneyu(gl, gh, alternative='two-sided')
            sig = '***' if pv < 0.001 else ('**' if pv < 0.01 else ('*' if pv < 0.05 else 'n.s.'))
            yb = ax.get_ylim()[1] * 0.88
            ax.plot([0, 0, 3, 3], [yb*0.95, yb, yb, yb*0.95], lw=0.5, color='black')
            ax.text(1.5, yb * 1.02, sig, ha='center', va='bottom', fontsize=6, fontweight='bold')
        add_panel_label(ax, label)

    legend_el = [mpatches.Patch(facecolor=income_colors[g], alpha=0.75,
                 edgecolor='#333333', linewidth=0.5, label=g) for g in income_order]
    fig.legend(handles=legend_el, loc='upper center', ncol=4, fontsize=6,
              frameon=False, bbox_to_anchor=(0.5, 0.99))
    save_figure(fig, 'Figure1_multihazard_exposure')
    plt.show()
    return fig

print("\nGenerating Figure 1...")
fig1 = figure1_multihazard(df_valid, INCOME_ORDER, INCOME_COLORS)

## Cell 7: Figure 2 — LECZ Exposure and Human Development
**Panel a:** Scatter plot of LECZ population share (< 10 m) versus HDI for 1,945 coastal urban centres, with LOWESS smoother and marginal histograms.  
**Panel b:** Box plots of OLS regression residuals (LECZ ~ HDI + log-population + log-area + SDG region fixed effects) by income group, testing whether income carries explanatory power beyond development and geography.  
Corresponds to **Figure 2** in the manuscript.

In [ ]:
def figure2_lecz_hdi(data, income_colors):
    """Figure 2: LECZ exposure vs HDI (panel a) + regression residuals (panel b)."""
    mask = (data['share_pop_lecz_below10m'].notna() &
            (data['share_pop_lecz_below10m'] > 0) & data['hdi'].notna())
    pdf = data[mask].copy()
    if len(pdf) < 50:
        print("  Warning: Too few cities for Figure 2"); return None

    fig = plt.figure(figsize=(NATURE_DOUBLE_COL, NATURE_DOUBLE_COL * 0.50))
    gs_left = gridspec.GridSpec(2, 2, width_ratios=[4, 1], height_ratios=[1, 4],
                                hspace=0.05, wspace=0.05,
                                left=0.07, right=0.52, top=0.92, bottom=0.12)
    ax_m = fig.add_subplot(gs_left[1, 0])
    ax_t = fig.add_subplot(gs_left[0, 0], sharex=ax_m)
    ax_r = fig.add_subplot(gs_left[1, 1], sharey=ax_m)

    gs_right = gridspec.GridSpec(1, 1, left=0.62, right=0.97, top=0.86, bottom=0.12)
    ax_b = fig.add_subplot(gs_right[0, 0])

    # Panel (a): LECZ scatter with LOWESS and marginal histograms
    for grp in INCOME_ORDER:
        gd = pdf[pdf['income_group'] == grp]
        if len(gd) == 0: continue
        sizes = np.clip(gd['population'] / 50000, 1, 40)
        ax_m.scatter(gd['hdi'], gd['share_pop_lecz_below10m'], s=sizes, alpha=0.45,
                    color=income_colors[grp], label=grp, edgecolors='none', rasterized=True)

    valid = pdf[['hdi', 'share_pop_lecz_below10m']].dropna()
    if len(valid) > 50:
        sm_out = lowess(valid['share_pop_lecz_below10m'], valid['hdi'], frac=0.3, return_sorted=True)
        ax_m.plot(sm_out[:, 0], sm_out[:, 1], color='black', lw=1.2, ls='--', zorder=5, alpha=0.8)

    ax_m.set_xlabel('Human Development Index (HDI)')
    ax_m.set_ylabel('Population in LECZ <10 m (%)')
    rho, pr = spearmanr(valid['hdi'], valid['share_pop_lecz_below10m'])
    pt = 'P < 0.001' if pr < 0.001 else f'P = {pr:.3f}'
    ax_m.text(0.97, 0.97, f'\u03c1 = {rho:.3f}\n{pt}\nn = {len(valid):,}',
             transform=ax_m.transAxes, fontsize=6, va='top', ha='right',
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white',
                      edgecolor='#ccc', alpha=0.9, linewidth=0.5))

    for grp in INCOME_ORDER:
        v = pdf.loc[pdf['income_group'] == grp, 'hdi'].dropna()
        if len(v) > 5: ax_t.hist(v, bins=30, alpha=0.5, color=income_colors[grp], density=True, edgecolor='none')
    ax_t.set_ylabel('Density', fontsize=5); ax_t.tick_params(labelbottom=False)

    for grp in INCOME_ORDER:
        v = pdf.loc[pdf['income_group'] == grp, 'share_pop_lecz_below10m'].dropna()
        if len(v) > 5: ax_r.hist(v, bins=30, alpha=0.5, color=income_colors[grp], density=True, orientation='horizontal', edgecolor='none')
    ax_r.set_xlabel('Density', fontsize=5); ax_r.tick_params(labelleft=False)

    leg = [Line2D([0], [0], marker='o', color='w', markerfacecolor=income_colors[g],
                  markersize=5, label=g, markeredgewidth=0) for g in INCOME_ORDER]
    ax_m.legend(handles=leg, loc='upper left', fontsize=5.5, frameon=True,
               fancybox=False, edgecolor='#ccc', framealpha=0.9)
    add_panel_label(ax_t, 'a', x=-0.10, y=1.15)

    # Panel (b): LECZ regression residuals by income group
    reg_cols = ['share_pop_lecz_below10m', 'hdi', 'population', 'area_km2',
                'sdg_region', 'income_group']
    rd = pdf[reg_cols].dropna().copy()
    rd['log_pop'] = np.log1p(rd['population'])
    rd['log_area'] = np.log1p(rd['area_km2'])

    if len(rd) >= 100:
        lecz_model = smf.ols(
            'share_pop_lecz_below10m ~ hdi + log_pop + log_area + C(sdg_region)',
            data=rd).fit(cov_type='HC3')
        rd['residual'] = lecz_model.resid

        print(f"\n  LECZ regression (panel b):")
        print(f"    N = {int(lecz_model.nobs):,}  |  R² = {lecz_model.rsquared:.4f}  "
              f"|  Adj. R² = {lecz_model.rsquared_adj:.4f}")

        box_data = [rd.loc[rd['income_group'] == grp, 'residual'].values
                    for grp in INCOME_ORDER]
        bp = ax_b.boxplot(box_data, patch_artist=True, widths=0.6,
                          showfliers=False, showmeans=True,
                          meanprops=dict(marker='D', markerfacecolor='white',
                                        markeredgecolor='#333', markersize=4),
                          medianprops=dict(color='#333', linewidth=1.0),
                          whiskerprops=dict(color='#555', linewidth=0.8),
                          capprops=dict(color='#555', linewidth=0.8))
        for patch, grp in zip(bp['boxes'], INCOME_ORDER):
            patch.set_facecolor(income_colors[grp])
            patch.set_alpha(0.8)
            patch.set_edgecolor('#333')
            patch.set_linewidth(0.6)

        ax_b.set_xticklabels(['Low', 'Lower\nMiddle', 'Upper\nMiddle', 'High'], fontsize=6)
        ax_b.set_ylabel('LECZ residual (%)')
        ax_b.set_xlabel('Income group')
        ax_b.axhline(0, color='black', linewidth=0.6, linestyle='--', alpha=0.6)

        groups_resid = [rd.loc[rd['income_group'] == g, 'residual'].dropna().values
                        for g in INCOME_ORDER if len(rd.loc[rd['income_group'] == g, 'residual'].dropna()) > 0]
        if len(groups_resid) >= 2:
            kw_h, kw_p = kruskal(*groups_resid)
            pt_kw = 'P < 0.001' if kw_p < 0.001 else f'P = {kw_p:.3f}'
            ax_b.text(0.97, 0.97, f'KW H = {kw_h:.1f}\n{pt_kw}',
                     transform=ax_b.transAxes, fontsize=6, va='top', ha='right',
                     bbox=dict(boxstyle='round,pad=0.3', facecolor='white',
                              edgecolor='#ccc', alpha=0.9, linewidth=0.5))

        for grp in INCOME_ORDER:
            gv = rd.loc[rd['income_group'] == grp, 'residual']
            print(f"    {grp:15s}: median residual = {gv.median():+.2f}  "
                  f"(IQR {gv.quantile(0.25):+.2f} to {gv.quantile(0.75):+.2f}, n = {len(gv):,})")
    else:
        ax_b.text(0.5, 0.5, 'Insufficient data\nfor regression',
                 transform=ax_b.transAxes, ha='center', va='center', fontsize=7)

    add_panel_label(ax_b, 'b', x=-0.15, y=1.06)
    save_figure(fig, 'Figure2_LECZ_vs_HDI')
    plt.show()
    return fig

print("\nGenerating Figure 2...")
fig2 = figure2_lecz_hdi(df_valid, INCOME_COLORS)

## Cell 8: Figure 3 — Hazard × HDI Bivariate Heatmap
Joint distribution of CHI quintiles and HDI quintiles showing (a) city counts and (b) mean LECZ population per cell. Corresponds to **Figure 3** in the manuscript.

In [ ]:
def figure3_heatmap(data):
    """Figure 3: Joint distribution of CHI and HDI quintiles."""
    mask = data['hdi_quintile'].notna() & data['hazard_quintile'].notna()
    sub = data[mask].copy()
    if len(sub) < 100:
        print("  Warning: Too few for Figure 3"); return None

    ct_n = pd.crosstab(sub['hazard_quintile'], sub['hdi_quintile'])
    ct_p = sub.groupby(['hazard_quintile', 'hdi_quintile'])['pop_lecz_total'].mean().unstack(fill_value=0)
    ct_p = ct_p.reindex(index=ct_n.index, columns=ct_n.columns, fill_value=0)

    fig, axes = plt.subplots(1, 2, figsize=(NATURE_DOUBLE_COL, NATURE_DOUBLE_COL * 0.4))

    cmap_a = LinearSegmentedColormap.from_list('c', ['#FFF', '#C6DBEF', '#6BAED6', '#2171B5', '#08306B'])
    im = axes[0].imshow(ct_n.values, cmap=cmap_a, aspect='auto')
    for i in range(ct_n.shape[0]):
        for j in range(ct_n.shape[1]):
            v = ct_n.values[i, j]
            axes[0].text(j, i, f'{v:,}', ha='center', va='center', fontsize=5,
                        color='white' if v > ct_n.values.max()*0.6 else 'black', fontweight='bold')
    axes[0].set_xticks(range(ct_n.shape[1])); axes[0].set_xticklabels(ct_n.columns, fontsize=5.5)
    axes[0].set_yticks(range(ct_n.shape[0])); axes[0].set_yticklabels(ct_n.index, fontsize=5.5)
    axes[0].set_xlabel('HDI quintile', fontsize=6.5)
    axes[0].set_ylabel('Compound Hazard\nExposure quintile', fontsize=6.5)
    cb = fig.colorbar(im, ax=axes[0], shrink=0.8, pad=0.03)
    cb.ax.tick_params(labelsize=5); cb.set_label('N urban centres', fontsize=5.5)
    add_panel_label(axes[0], 'a', x=-0.2, y=1.12)

    cmap_b = LinearSegmentedColormap.from_list('p', ['#FFF', '#FEE0D2', '#FC9272', '#DE2D26', '#A50F15'])
    im2 = axes[1].imshow(ct_p.values, cmap=cmap_b, aspect='auto')
    for i in range(ct_p.shape[0]):
        for j in range(ct_p.shape[1]):
            v = ct_p.values[i, j]
            lb = f'{v/1000:.1f}k' if v >= 1000 else f'{v:.0f}'
            axes[1].text(j, i, lb, ha='center', va='center', fontsize=5,
                        color='white' if v > ct_p.values.max()*0.5 else 'black', fontweight='bold')
    axes[1].set_xticks(range(ct_p.shape[1])); axes[1].set_xticklabels(ct_p.columns, fontsize=5.5)
    axes[1].set_yticks(range(ct_p.shape[0])); axes[1].set_yticklabels(ct_p.index, fontsize=5.5)
    axes[1].set_xlabel('HDI quintile', fontsize=6.5); axes[1].set_ylabel('')
    cb2 = fig.colorbar(im2, ax=axes[1], shrink=0.8, pad=0.03)
    cb2.ax.tick_params(labelsize=5); cb2.set_label('Mean LECZ pop.', fontsize=5.5)
    add_panel_label(axes[1], 'b', x=-0.15, y=1.12)

    plt.tight_layout(w_pad=2)
    save_figure(fig, 'Figure3_hazard_HDI_heatmap')
    plt.show()
    return fig

print("\nGenerating Figure 3...")
fig3 = figure3_heatmap(df_valid)

## Cell 9: Figure 4 — Demographic Composition
**Panels a–b:** Mean youth (0–14) and elderly (>65) population shares in cities below vs above median CHI, stratified by income group.  
**Panel c:** Youth-to-elderly ratio across hazard quintiles by income group.  
Corresponds to **Figure 4** in the manuscript.

In [ ]:
def figure4_demographics(data, income_order, income_colors):
    """Figure 4: Demographic composition across exposure and income strata."""
    med_haz = data['compound_hazard_index'].median()
    data = data.copy()
    data['high_exp'] = data['compound_hazard_index'] >= med_haz

    fig, axes = plt.subplots(1, 3, figsize=(NATURE_DOUBLE_COL, NATURE_DOUBLE_COL * 0.33))
    bw = 0.35
    x = np.arange(len(income_order))

    for ai, (ax, var, yl) in enumerate(zip(
        axes[:2], ['pct_young', 'pct_old'],
        ['Population aged\n0\u201314 (%)', 'Population aged\n>65 (%)']
    )):
        if var not in data.columns:
            ax.text(0.5, 0.5, 'N/A', transform=ax.transAxes, ha='center', fontsize=6)
            add_panel_label(ax, chr(97+ai)); continue
        for i, (el, ev) in enumerate([('Below median\nexposure', False),
                                       ('Above median\nexposure', True)]):
            means, cis = [], []
            for grp in income_order:
                v = data.loc[(data['income_group']==grp) & (data['high_exp']==ev), var].dropna()
                means.append(v.mean() if len(v) > 0 else 0)
                cis.append(1.96 * v.std()/np.sqrt(len(v)) if len(v) > 1 else 0)
            off = -bw/2 + i*bw
            col = WONG_PALETTE['sky_blue'] if not ev else WONG_PALETTE['vermilion']
            ax.bar(x+off, means, bw*0.9, yerr=cis, label=el, color=col, alpha=0.8,
                   edgecolor='white', linewidth=0.3, error_kw=dict(lw=0.5, capsize=2, capthick=0.5))
        short_labels = ['Low', 'Lower\nMiddle', 'Upper\nMiddle', 'High']
        ax.set_xticks(x); ax.set_xticklabels(short_labels, fontsize=5.5)
        ax.set_ylabel(yl)
        ax.legend(fontsize=5, loc='upper right' if ai==0 else 'upper left')
        add_panel_label(ax, chr(97+ai), x=-0.18, y=1.1)

    ax = axes[2]
    if 'pct_young' in data.columns and 'pct_old' in data.columns:
        data['yer'] = data['pct_young'] / data['pct_old'].replace(0, np.nan)
        for grp in income_order:
            gd = data[data['income_group'] == grp].copy()
            try:
                gd['hq'] = pd.qcut(gd['compound_hazard_index'], q=5, labels=[1,2,3,4,5], duplicates='drop')
                m = gd.groupby('hq')['yer'].mean()
                ax.plot(m.index.astype(int), m.values, marker='o', markersize=3, lw=0.8,
                       color=income_colors[grp], label=grp)
            except Exception: continue
        ax.set_xlabel('Hazard quintile', fontsize=6)
        ax.set_ylabel('Youth-to-elderly\nratio')
        ax.set_xticks([1,2,3,4,5]); ax.set_xticklabels(['Q1','Q2','Q3','Q4','Q5'], fontsize=5.5)
        ax.legend(fontsize=5, loc='upper left')
    add_panel_label(ax, 'c', x=-0.18, y=1.1)
    plt.tight_layout(w_pad=1.5)
    save_figure(fig, 'Figure4_demographics')
    plt.show()
    return fig

print("\nGenerating Figure 4...")
fig4 = figure4_demographics(df_valid, INCOME_ORDER, INCOME_COLORS)

## Cell 10: Figure 5 — Quantile Regression
Quantile regression coefficients (τ = 0.10 to 0.90) for income group effects on CHI, with OLS comparison. Demonstrates that the income–exposure gradient is concentrated at the upper tail of the hazard distribution. Corresponds to **Figure 5** in the manuscript.

In [ ]:
def figure5_quantile_regression(data, income_order):
    """Figure 5: Quantile regression coefficients for income group effects on CHI."""
    rd = data[['compound_hazard_index', 'income_group', 'area_km2', 'population']].dropna().copy()
    if len(rd) < 100:
        print("  Too few obs"); return None, {}

    rd['log_pop'] = np.log1p(rd['population'])
    rd['log_area'] = np.log1p(rd['area_km2'])
    rd = pd.get_dummies(rd, columns=['income_group'], drop_first=False, dtype=float)

    dummies = [f'income_group_{g}' for g in income_order if g != 'High'
               and f'income_group_{g}' in rd.columns]
    if not dummies:
        print("  No income dummies"); return None, {}

    X = sm.add_constant(rd[dummies + ['log_pop', 'log_area']].astype(float))
    y = rd['compound_hazard_index'].astype(float)

    quantiles = [0.10, 0.25, 0.50, 0.75, 0.90]
    results = {q: {} for q in quantiles}
    print("  Quantile regressions...")
    for q in quantiles:
        try:
            res = QuantReg(y, X).fit(q=q, max_iter=5000)
            for d in dummies:
                gl = d.replace('income_group_', '')
                results[q][gl] = {'coef': res.params[d],
                    'ci_low': res.conf_int().loc[d, 0],
                    'ci_high': res.conf_int().loc[d, 1]}
            print(f"    \u03c4={q:.2f} done")
        except Exception as e:
            print(f"    \u03c4={q:.2f} failed: {e}")

    ols = sm.OLS(y, X).fit(cov_type='HC3')

    grps = [g for g in ['Low', 'Lower Middle', 'Upper Middle']
            if f'income_group_{g}' in rd.columns]
    fig, axes = plt.subplots(1, len(grps),
                              figsize=(NATURE_DOUBLE_COL, NATURE_DOUBLE_COL*0.3), sharey=True)
    if len(grps) == 1: axes = [axes]

    for idx, grp in enumerate(grps):
        ax = axes[idx]
        col = INCOME_COLORS.get(grp, '#333')
        coefs = [results[q].get(grp, {}).get('coef', np.nan) for q in quantiles]
        cil = [results[q].get(grp, {}).get('ci_low', np.nan) for q in quantiles]
        cih = [results[q].get(grp, {}).get('ci_high', np.nan) for q in quantiles]
        ax.fill_between(quantiles, cil, cih, alpha=0.2, color=col)
        ax.plot(quantiles, coefs, 'o-', color=col, markersize=3.5, lw=1,
               markeredgecolor='white', markeredgewidth=0.3)
        dn = f'income_group_{grp}'
        if dn in ols.params.index:
            ax.axhline(ols.params[dn], color='#999', ls=':', lw=0.6,
                      label=f'OLS: {ols.params[dn]:.4f}')
        ax.axhline(0, color='black', lw=0.4)
        ax.set_xlabel('Quantile (\u03c4)')
        ax.set_title(f'{grp}\n(ref: High)', fontsize=6.5, fontweight='bold')
        ax.set_xticks(quantiles)
        ax.set_xticklabels([f'{q:.2f}' for q in quantiles], fontsize=5)
        if idx == 0: ax.set_ylabel('Coefficient (CHI)')
        ax.legend(fontsize=4.5, loc='best')
        add_panel_label(ax, chr(97+idx), x=-0.15, y=1.15)

    plt.tight_layout(w_pad=1)
    save_figure(fig, 'Figure5_quantile_regression')
    plt.show()
    return fig, results

print("\nGenerating Figure 5...")
fig5, qreg = figure5_quantile_regression(df_valid, INCOME_ORDER)

## Cell 11: OLS Regression Table
Four model specifications with HC3 heteroskedasticity-consistent standard errors:
- Model 1: Bivariate (income group only)
- Model 2: + city size controls (log-population, log-area)
- Model 3: + SDG region fixed effects
- Model 4: HDI (continuous) + city size controls

Corresponds to **Extended Data Table 1** in the manuscript.

In [ ]:
def run_regressions(data):
    """Extended Data Table 1: OLS regression results (Models 1-4)."""
    rd = data[['compound_hazard_index', 'income_group', 'sdg_region',
               'area_km2', 'population', 'hdi']].dropna().copy()
    if len(rd) < 100:
        print("  Too few obs"); return {}
    rd['log_pop'] = np.log1p(rd['population'])
    rd['log_area'] = np.log1p(rd['area_km2'])

    m1 = smf.ols('compound_hazard_index ~ C(income_group, Treatment(reference="High"))',
                  data=rd).fit(cov_type='HC3')
    m2 = smf.ols('compound_hazard_index ~ C(income_group, Treatment(reference="High"))'
                  ' + log_pop + log_area', data=rd).fit(cov_type='HC3')
    m3 = smf.ols('compound_hazard_index ~ C(income_group, Treatment(reference="High"))'
                  ' + log_pop + log_area + C(sdg_region)', data=rd).fit(cov_type='HC3')
    m4 = smf.ols('compound_hazard_index ~ hdi + log_pop + log_area',
                  data=rd).fit(cov_type='HC3')

    print("\n" + "="*80)
    print("REGRESSION RESULTS")
    print("="*80)
    from statsmodels.iolib.summary2 import summary_col
    s = summary_col([m1, m2, m3, m4],
        model_names=['(1) Bivariate', '(2) Controls', '(3) Region FE', '(4) HDI'],
        stars=True, info_dict={
            'N': lambda x: f"{int(x.nobs):,}",
            'R\u00b2': lambda x: f"{x.rsquared:.4f}",
            'Adj. R\u00b2': lambda x: f"{x.rsquared_adj:.4f}"})
    print(s)
    s.tables[0].to_csv('tables/regression_results.csv')
    print("  Saved: tables/regression_results.csv")
    return {'m1': m1, 'm2': m2, 'm3': m3, 'm4': m4}

print("\nRunning regressions...")
reg_models = run_regressions(df_valid)

## Cell 12: Extended Data — Regional Disaggregation
Summary statistics for CHI, flood exposure, LECZ, HDI, and demographics by SDG region. Supports **Extended Data Figure 1** in the manuscript.

In [ ]:
def extended_regional(data):
    """Extended Data: Regional disaggregation of key variables."""
    regions = sorted(data['sdg_region'].dropna().unique())
    vm = {'compound_hazard_index': 'CHI', 'share_pop_flood_100yr': 'Flood',
          'share_pop_lecz_below10m': 'LECZ', 'hdi': 'HDI',
          'pct_young': 'Young', 'pct_old': 'Old'}
    vm = {k: v for k, v in vm.items() if k in data.columns}
    rows = []
    for r in regions:
        rd = data[data['sdg_region'] == r]
        row = {'Region': r, 'N': rd.shape[0]}
        for var, lbl in vm.items():
            v = rd[var].dropna()
            if len(v) > 0:
                row[f'{lbl} med'] = f"{v.median():.3f}"
                row[f'{lbl} IQR'] = f"{v.quantile(0.25):.3f}\u2013{v.quantile(0.75):.3f}"
            else:
                row[f'{lbl} med'] = '\u2014'; row[f'{lbl} IQR'] = '\u2014'
        rows.append(row)
    ed = pd.DataFrame(rows)
    ed.to_csv('tables/ExtendedData_regional.csv', index=False)
    ed.to_excel('tables/ExtendedData_regional.xlsx', index=False)
    print("  Saved: tables/ExtendedData_regional.csv/.xlsx")
    return ed

print("\nExtended Data: Regional...")
ed_reg = extended_regional(df_valid)
print(ed_reg[['Region', 'N']].to_string(index=False))

## Cell 13: Top 50 Exposed Low-Income Cities
Profiles of the 50 most exposed urban centres classified as Low or Lower Middle income. Corresponds to **Extended Data Table 3** in the manuscript.

In [ ]:
def top50_exposed(data):
    """Extended Data Table 3: Top 50 most exposed low-income urban centres."""
    low = data[data['income_group'].isin(['Low', 'Lower Middle'])]
    top = low.nlargest(50, 'compound_hazard_index')
    cols = ['name', 'country_gadm', 'income_group', 'population', 'hdi',
            'compound_hazard_index', 'share_pop_flood_100yr',
            'share_pop_lecz_below10m', 'share_pop_earthquake_mmi6', 'pct_young', 'pct_old']
    avail = [c for c in cols if c in top.columns]
    result = top[avail].reset_index(drop=True)
    result.index = result.index + 1; result.index.name = 'Rank'
    result.to_csv('tables/ExtendedData_top50.csv')
    result.to_excel('tables/ExtendedData_top50.xlsx')
    print("  Saved: tables/ExtendedData_top50.csv/.xlsx")
    return result

print("\nTop 50 exposed low-income cities...")
top50 = top50_exposed(df_valid)
print(top50.head(10).to_string())

## Cell 14: Robustness Checks
Sensitivity analyses including:
1. Population thresholds (≥50k, ≥100k, ≥500k)
2. GDP per capita quintile classification
3. LECZ threshold (5 m vs 10 m)
4. Geometric mean vs arithmetic mean CHI construction

Corresponds to **Extended Data Table 2** in the manuscript.

In [ ]:
def robustness_checks(data, income_order):
    """Extended Data Table 2: Sensitivity analyses."""
    print("\n" + "="*80)
    print("ROBUSTNESS CHECKS")
    print("="*80)

    # 1. Population thresholds
    print("\n--- Check 1: Population >= 50,000 ---")
    sub = data[data['population'] >= 50000]
    print(f"  N = {sub.shape[0]:,}")
    for g in income_order:
        v = sub.loc[sub['income_group']==g, 'compound_hazard_index'].dropna()
        if len(v) > 0: print(f"  {g}: median CHI = {v.median():.4f}")
    vg = [sub.loc[sub['income_group']==g, 'compound_hazard_index'].dropna() for g in income_order]
    vg = [g for g in vg if len(g) >= 5]
    if len(vg) >= 2:
        h, p = kruskal(*vg); print(f"  KW: H={h:.2f}, p={p:.2e}")

    # 2. GDP quintiles
    print("\n--- Check 2: GDP quintiles ---")
    if 'gdp_avg' in data.columns:
        sub = data[data['gdp_avg'].notna()].copy()
        try:
            sub['gq'] = pd.qcut(sub['gdp_avg'], q=5, labels=['Q1','Q2','Q3','Q4','Q5'], duplicates='drop')
            for q in sub['gq'].cat.categories:
                v = sub.loc[sub['gq']==q, 'compound_hazard_index'].dropna()
                print(f"  {q}: median CHI = {v.median():.4f}, n={len(v):,}")
        except Exception as e: print(f"  Skipped: {e}")

    # 3. LECZ 5m threshold
    print("\n--- Check 3: LECZ 5m ---")
    if 'share_pop_lecz_5m' in data.columns:
        for g in income_order:
            v = data.loc[data['income_group']==g, 'share_pop_lecz_5m'].dropna()
            if len(v) > 0: print(f"  {g}: median = {v.median():.2f}%")

    # 4. Geometric mean CHI
    print("\n--- Check 4: Geometric mean ---")
    nc = [c for c in data.columns if c.endswith('_norm')]
    if nc:
        eps = 1e-6
        dc = data.copy()
        dc['chi_geo'] = np.exp(dc[nc].apply(lambda x: np.log(x + eps)).mean(axis=1))
        vp = dc[['compound_hazard_index', 'chi_geo']].dropna()
        if len(vp) > 50:
            rho, p = spearmanr(vp['compound_hazard_index'], vp['chi_geo'])
            print(f"  Spearman rho = {rho:.4f}, p = {p:.2e}")

robustness_checks(df_valid, INCOME_ORDER)

## Cell 15: Extended Data Figure 3 — Spearman Correlation Matrix
Pairwise Spearman rank correlations across all exposure, socioeconomic, demographic, infrastructure, and environmental variables. Corresponds to **Extended Data Figure 3** in the manuscript.

In [ ]:
def extended_correlation(data):
    """Extended Data Figure 3: Spearman rank correlation matrix."""
    cv = {'compound_hazard_index': 'Compound Hazard',
          'share_pop_flood_100yr': 'Flood', 'share_pop_lecz_below10m': 'LECZ',
          'share_pop_coastal_100yr': 'Coastal', 'share_pop_earthquake_mmi6': 'Earthquake',
          'hdi': 'HDI', 'gdp_avg': 'GDP p.c.', 'life_exp': 'Life Exp.',
          'pct_young': 'Young', 'pct_old': 'Elderly', 'pct_female': 'Female',
          'pm25_conc': 'PM2.5', 'cisi_all': 'CISI'}
    av = {k: v for k, v in cv.items() if k in data.columns}
    sub = data[list(av.keys())].dropna()
    if len(sub) < 50:
        print("  Too few obs for correlation"); return None

    corr = sub.corr(method='spearman')
    corr.index = [av[c] for c in corr.index]
    corr.columns = [av[c] for c in corr.columns]

    fig, ax = plt.subplots(figsize=(NATURE_DOUBLE_COL, NATURE_DOUBLE_COL * 0.65))
    cmap = LinearSegmentedColormap.from_list('d',
        [WONG_PALETTE['blue'], '#FFFFFF', WONG_PALETTE['vermilion']], N=256)
    mask = np.triu(np.ones_like(corr, dtype=bool), k=1)
    sns.heatmap(corr, mask=mask, cmap=cmap, center=0, vmin=-1, vmax=1,
                square=True, linewidths=0.3, linecolor='white',
                annot=True, fmt='.2f', annot_kws={'size': 5},
                cbar_kws={'shrink': 0.7, 'label': "Spearman's rho"}, ax=ax)
    ax.tick_params(labelsize=5.5)
    plt.tight_layout()
    save_figure(fig, 'ExtendedData_correlation_matrix')
    plt.show()
    return fig

print("\nCorrelation matrix...")
fig_corr = extended_correlation(df_valid)

## Cell 16: Analysis Summary
Output file inventory and completion confirmation.

In [ ]:
print("\n" + "="*80)
print("ANALYSIS COMPLETE")
print("="*80)
print(f"""
Outputs:
  figures/  Figure1-5 + ExtendedData_correlation_matrix (.pdf/.png)
  tables/   Table1, regression, regional, top50 (.csv/.xlsx)

Dataset: GHS-UCDB R2024A V1.1 (GeoPackage)
  DOI: 10.2905/1a338be6-7eaf-480c-9664-3a8ade88cbcd
""")